In [14]:
import socket
import json
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import tensorflow as tf
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Bidirectional,BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score

In [15]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, holistic_model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = holistic_model.process(image)        # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def extract_left_keypoints(results):
    # Trích xuất keypoints của bàn tay trái (left_hand_landmarks)
    if results.left_hand_landmarks:
        lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark])
    else:
        lh = np.zeros((21, 2))  # shape (21, 2) vì mỗi keypoint có 2 giá trị (x, y)
    return lh

def extract_index_keypoints(results, image):
    # Trích xuất tọa độ x, y của ngón trỏ (index 8)
    if results.left_hand_landmarks:
        # Lấy tọa độ ngón trỏ (điểm mốc thứ 8)
        index_finger = results.left_hand_landmarks.landmark[8]
        return np.array([index_finger.x, index_finger.y])  # Tọa độ x, y của ngón trỏ
    else:
        # Nếu không phát hiện tay trái, trả về [0, 0]
        return np.zeros(2)


def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [16]:
classes_direction = ['idle', 'up', 'down', 'left', 'right']

label_map = {label:num for num, label in enumerate(classes_direction)}
label_map

{'idle': 0, 'up': 1, 'down': 2, 'left': 3, 'right': 4}

In [17]:
classes_movement = ['movement', 'other']

label_map = {label:num for num, label in enumerate(classes_movement)}
label_map

{'movement': 0, 'other': 1}

In [18]:
def normalize_points(points):
    origin = points[0]
    
    # Chuẩn hóa tọa độ bằng cách trừ tọa độ frame đầu tiên
    normalized_points = points - origin
    
    return normalized_points

In [19]:
def normalize_keypoints(keypoints):
    # Kiểm tra nếu có đủ số điểm (21 điểm cho mỗi bàn tay)
    if keypoints.shape[0] != 21:
        raise ValueError(f"Số lượng điểm keypoints không hợp lệ: {keypoints.shape[0]}")

    # Cổ tay là điểm đầu tiên trong keypoints (index 0)
    wrist = keypoints[0]
    
    # Dịch các điểm sao cho cổ tay trở thành gốc tọa độ (0, 0)
    normalized_keypoints = []
    for point in keypoints:
        normalized_point = (point[0] - wrist[0], point[1] - wrist[1])  # Chỉ cần dịch x, y
        normalized_keypoints.append(normalized_point)
    
    # Chuyển sang numpy array để dễ dàng tính toán min và max
    normalized_keypoints = np.array(normalized_keypoints)
    
    # Tính toán min và max cho x và y
    x_min, y_min = np.min(normalized_keypoints, axis=0)
    x_max, y_max = np.max(normalized_keypoints, axis=0)
    
    # Tránh chia cho 0 nếu max - min = 0
    if (x_max - x_min) == 0:
        print("Cảnh báo: Tọa độ x không thay đổi, bỏ qua chuẩn hóa x.")
        x_min, x_max = 0, 1  # Cứ để giá trị x giữ nguyên, hoặc chọn giá trị mặc định
    if (y_max - y_min) == 0:
        print("Cảnh báo: Tọa độ y không thay đổi, bỏ qua chuẩn hóa y.")
        y_min, y_max = 0, 1  # Cứ để giá trị y giữ nguyên, hoặc chọn giá trị mặc định
    
    # Chuyển min và max về dạng numpy array để có thể tính toán đúng
    min_vals = np.array([x_min, y_min])
    max_vals = np.array([x_max, y_max])
    
    # Chuẩn hóa về phạm vi [-1, 1]
    normalized_keypoints = 2 * (normalized_keypoints - min_vals) / (max_vals - min_vals) - 1
    
    return normalized_keypoints

In [20]:
model_hand = tf.keras.models.load_model('movement_check.keras')
model_hand.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 21, 128)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2688)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2688)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │       172,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 517,637 (1.97 MB)

 Trainable params: 172,545 (674.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 345,092 (1.32 MB)

In [21]:
model_pointer = tf.keras.models.load_model('movement_direction.keras')
model_pointer.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 15, 64)         │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 379,409 (1.45 MB)

 Trainable params: 126,469 (494.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 252,940 (988.05 KB)

In [22]:
model_hand.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model_pointer.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,234,245), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [24]:
IP = '127.0.0.1'
PORT = 25001

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def SendData(message):
    s.sendto(message.encode(), (IP, PORT))
    print(f"{message} sent")

In [35]:
# 1. New detection variables
keys = []
sentence = []
predictions = 0
threshold = 0.7
frame_counter = 0  # Khởi tạo biến đếm
isMovement = 0

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        frame_counter += 1  # Tăng biến đếm lên mỗi khi đọc khung hình
        if frame_counter % 3 != 0:  # Chỉ xử lý mỗi khung hình thứ 3 (tùy chỉnh)
            continue

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Check if the movement gesture is on
        if frame_counter % 30 == 0:
            hand_keypoints = extract_left_keypoints(results)
            hand_keypoints = normalize_keypoints(hand_keypoints)
            hand_keypoints = np.expand_dims(hand_keypoints, axis=0)
            res = model_hand.predict(hand_keypoints)
            if res[0] > 0.5:
                isMovement = 0
            else: isMovement = 1
            
        # Kiểm tra dự đoán
        if isMovement == 0:
            print('Other')
        else:
            print('Movement')
            index_keypoints = extract_index_keypoints(results, image)
            keys.append(index_keypoints)
            sequence = np.array(keys[-15:])
            sequence = normalize_points(sequence)
            
            if len(sequence) == 15:
                res = model_pointer.predict(np.expand_dims(sequence, axis=0))[0]
                print(np.argmax(res))
                print(classes_direction[np.argmax(res)])
                predictions = np.argmax(res)

                # Viz probabilities
                image = prob_viz(res, classes_direction, image, colors)
        
        # Send data through UDP
        send_result = f"{isMovement}, {predictions}"
        SendData(send_result)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Cảnh báo: Tọa độ x không thay đổi, bỏ qua chuẩn hóa x.
Cảnh báo: Tọa độ y không thay đổi, bỏ qua chuẩn hóa y.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Other
0, 0 sent
<class 'mediapipe.python.solution_base.SolutionOu